In [2]:
import pandas as pd
import numpy as np
import re

In [2]:
df_raw = pd.read_csv('../Data/drug_data.csv', index_col=0)
df_raw.head()

,CASEID,DRUGID_1,CATID_1_1,CATID_2_1,CATID_3_1,ROUTE_1,TOXTEST_1,sdled_1_1,sdled_2_1,sdled_3_1,...,QUARTER,DAYPART,NUMSUBS,CASETYPE,DISPOSITION,ALLABUSE,METRO,AGECAT,SEX,RACE
0,1,865,105,110,2005,1,2,1,2.0,-7.00,...,4,1,2,8,6,1,2,4,1,2
1,2,2077,81,82,283,-8,2,17,67.0,67.01,...,3,4,1,4,7,0,2,11,1,3
2,3,2313,1,12,-7,-8,2,17,64.0,64.99,...,4,3,1,4,1,0,7,11,2,2
3,4,234,358,99,215,2,2,17,73.0,73.01,...,2,2,1,4,7,0,10,2,1,3
4,5,865,105,110,2005,1,2,1,2.0,-7.00,...,3,4,3,8,8,1,1,6,1,3


Find sdleds to drop

In [5]:
sdled_cols = [column for column in df_raw.columns if column.startswith('sdled')]
sdled_cols

['sdled_1_1',
 'sdled_2_1',
 'sdled_3_1',
 'sdled_4_1',
 'sdled_5_1',
 'sdled_6_1',
 'sdled_1_2',
 'sdled_2_2',
 'sdled_3_2',
 'sdled_4_2',
 'sdled_5_2',
 'sdled_6_2',
 'sdled_1_3',
 'sdled_2_3',
 'sdled_3_3',
 'sdled_4_3',
 'sdled_5_3',
 'sdled_6_3']

In [6]:
def pct_na(col):
    freq = col.value_counts()
    if -7 in freq.index:
        return (freq[-7] / df_raw.shape[0]) * 100
    else:
        return 0

df_sdled_na_pct = df_raw[sdled_cols].apply(lambda x: pct_na(x), axis=0)
df_sdled_na_pct

sdled_1_1     0.000000
sdled_2_1     0.000000
sdled_3_1    22.422471
sdled_4_1    48.588262
sdled_5_1    74.691025
sdled_6_1    92.030601
sdled_1_2    67.200274
sdled_2_2    67.200274
sdled_3_2    69.005252
sdled_4_2    79.985842
sdled_5_2    88.550811
sdled_6_2    96.789221
sdled_1_3    92.133821
sdled_2_3    92.133821
sdled_3_3    92.504681
sdled_4_3    94.554921
sdled_5_3    96.680977
sdled_6_3    99.153231
dtype: float64

In [7]:
sdleds_todrop = [col for col in sdled_cols if re.match(r'^sdled_[5-6]', col)]

In [8]:
df_new = df_raw.drop(columns = sdleds_todrop)
df_new.columns

Index(['CASEID', 'DRUGID_1', 'CATID_1_1', 'CATID_2_1', 'CATID_3_1', 'ROUTE_1',
       'TOXTEST_1', 'sdled_1_1', 'sdled_2_1', 'sdled_3_1', 'sdled_4_1',
       'DRUGID_2', 'CATID_1_2', 'CATID_2_2', 'CATID_3_2', 'ROUTE_2',
       'TOXTEST_2', 'sdled_1_2', 'sdled_2_2', 'sdled_3_2', 'sdled_4_2',
       'DRUGID_3', 'CATID_1_3', 'CATID_2_3', 'CATID_3_3', 'ROUTE_3',
       'TOXTEST_3', 'sdled_1_3', 'sdled_2_3', 'sdled_3_3', 'sdled_4_3',
       'ALCOHOL', 'NONALCILL', 'PHARMA', 'NONMEDPHARMA', 'CASEWGT', 'YEAR',
       'QUARTER', 'DAYPART', 'NUMSUBS', 'CASETYPE', 'DISPOSITION', 'ALLABUSE',
       'METRO', 'AGECAT', 'SEX', 'RACE'],
      dtype='object')

Find catid's to drop

In [37]:
catid_cols = [column for column in df_raw.columns if column.startswith('CATID')]
catid_cols

['CATID_1_1',
 'CATID_2_1',
 'CATID_3_1',
 'CATID_1_2',
 'CATID_2_2',
 'CATID_3_2',
 'CATID_1_3',
 'CATID_2_3',
 'CATID_3_3']

In [41]:
def pct_na(col):
    freq = col.value_counts()
    if -7 in freq.index:
        return (freq[-7] / df_raw.shape[0]) * 100
    else:
        return 0

df_catid_na_pct = df_raw[catid_cols].apply(lambda x: pct_na(x), axis=0)
df_catid_na_pct

CATID_1_1     0.000000
CATID_2_1     3.730532
CATID_3_1    21.995433
CATID_1_2    67.200274
CATID_2_2    68.936744
CATID_3_2    72.807034
CATID_1_3    92.133821
CATID_2_3    92.481845
CATID_3_3    93.461064
dtype: float64

Value maps for Top drugs in DRUGID 1, 2, and 3

In [32]:
top_drugids = {
    'DRUGID_1': [1255, 1254, 1253, 865, 2420, 21],
    'DRUGID_2': [1255, 1254],
    'DRUGID_3': [1255, 1254]
}

In [33]:
def find_related_list(drugid_col: str, drugids: list):
    drugid_map = {}
    for drugid in drugids:
        df_1 = df_new[df_new[drugid_col]==drugid]
        related_cols = [col for col in df_new.columns if (
            col.startswith(('CATID', 'sdled')) and col.endswith('_1'))]
        col_dict = {}
        for col in related_cols:
            col_dict[col] = df_1[col].unique()
        drugid_map[drugid] = col_dict
    
    return drugid_map

In [37]:
final_dict = {}
for did in top_drugids.keys():
    map_name = f'{did}_map'
    final_dict[map_name] = find_related_list(drugid_col=did, drugids=top_drugids[did])

In [38]:
final_dict

{'DRUGID_1_map': {1255: {'CATID_1_1': array([105]),
   'CATID_2_1': array([114]),
   'CATID_3_1': array([2032]),
   'sdled_1_1': array([1]),
   'sdled_2_1': array([2.5]),
   'sdled_3_1': array([5.]),
   'sdled_4_1': array([5.01])},
  1254: {'CATID_1_1': array([105]),
   'CATID_2_1': array([114]),
   'CATID_3_1': array([2032]),
   'sdled_1_1': array([1]),
   'sdled_2_1': array([2.5]),
   'sdled_3_1': array([3.]),
   'sdled_4_1': array([-7.])},
  1253: {'CATID_1_1': array([105]),
   'CATID_2_1': array([114]),
   'CATID_3_1': array([2032]),
   'sdled_1_1': array([1]),
   'sdled_2_1': array([2.5]),
   'sdled_3_1': array([4.]),
   'sdled_4_1': array([-7.])},
  865: {'CATID_1_1': array([105]),
   'CATID_2_1': array([110]),
   'CATID_3_1': array([2005]),
   'sdled_1_1': array([1]),
   'sdled_2_1': array([2.]),
   'sdled_3_1': array([-7.]),
   'sdled_4_1': array([-7.])},
  2420: {'CATID_1_1': array([2006]),
   'CATID_2_1': array([-7]),
   'CATID_3_1': array([-7]),
   'sdled_1_1': array([17]),


Get demographic dataset


In [3]:
df_raw = pd.read_csv('../Data/drug_data.csv', index_col=0)
demographic_df = df_raw[['CASEID', 'METRO', 'AGECAT', 'SEX', 'RACE','DISPOSITION', 'ALLABUSE', 'CASETYPE']]
demographic_df.replace({-7:0,-8:0,-9:0},inplace=True)
demographic_df.head()

/tmp/ipykernel_4355/772361444.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographic_df.replace({-7:0,-8:0,-9:0},inplace=True)


,CASEID,METRO,AGECAT,SEX,RACE,DISPOSITION,ALLABUSE,CASETYPE
0,1,2,4,1,2,6,1,8
1,2,2,11,1,3,7,0,4
2,3,7,11,2,2,1,0,4
3,4,10,2,1,3,7,0,4
4,5,1,6,1,3,8,1,8


In [4]:
demographic_df.to_csv('../Data/demographic_data.csv')

In [4]:
df_grouped = demographic_df.groupby(['AGECAT', 'CASETYPE']).agg(
    count_col=pd.NamedAgg(column='CASETYPE', aggfunc="count")
        )

In [5]:
df_grouped

count_col
AGECAT CASETYPE           
0      1                 2
       2                 2
       4                20
       5                 3
       6                 1
...                    ...
11     4             23212
       5              2034
       6                12
       7               262
       8              1496

[87 rows x 1 columns]

In [6]:
agecat_map = {0:'NA', 1:'<= 5', 2:'6-11',3:'12-17', 4:'18-20', 5:'21-24', 6:'25-29', 7:'30-34', 8:'35-44', 9:'45-54', 10:'55-64',11:'>= 65'}
sex_map = {0:'NA', 1:'Male', 2:'Female'}
race_map = {0:'NA', 1:'White', 2:'Black/African American',3:'Any Hispanic/Latino', 4:'All Other'}
casetype_map = {0: 'NA', 1:'Suicide Attempt', 2:'Seeking Detox',3:'Alcohol Only(Age<21)', 4:'Adverse Reaction', 5:'Overmedication', 6:'Malicious Poisoning', 7:'Accidental Injestion', 8:'Other'}
metro_map = {0:'NA', 1:'MA-NH', 2:'NY-NJ-PA',3:'IL-IN-WI', 4:'MI', 5:'MN-WI', 6:'FL', 7:'FL', 8:'TX', 9:'CO', 10:'AZ',11:'CA', 12:'CA', 13:'WA', 14:'Other'}
index_0 = df_grouped.index.get_level_values(0).map(agecat_map)
index_1 = df_grouped.index.get_level_values(1).map(casetype_map)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [7]:
df_grouped_mapped = df_grouped.set_index([index_0, index_1])

In [8]:
df_grouped_mapped

count_col
AGECAT CASETYPE                       
NA     Suicide Attempt               2
       Seeking Detox                 2
       Adverse Reaction             20
       Overmedication                3
       Malicious Poisoning           1
...                                ...
>= 65  Adverse Reaction          23212
       Overmedication             2034
       Malicious Poisoning          12
       Accidental Injestion        262
       Other                      1496

[87 rows x 1 columns]

In [21]:
df_grouped_mapped['count_col'].groupby('AGECAT', group_keys=False).nlargest(3)

AGECAT  CASETYPE            
12-17   Other                    4383
        Adverse Reaction         2557
        Alcohol Only(Age<21)     2433
18-20   Other                    5901
        Alcohol Only(Age<21)     4953
        Adverse Reaction         3070
21-24   Other                    9067
        Adverse Reaction         4331
        Seeking Detox            1490
25-29   Other                   10821
        Adverse Reaction         5595
        Seeking Detox            1750
30-34   Other                    9725
        Adverse Reaction         5141
        Seeking Detox            1561
35-44   Other                   17506
        Adverse Reaction         9988
        Seeking Detox            3099
45-54   Other                   17712
        Adverse Reaction        12035
        Seeking Detox            3513
55-64   Adverse Reaction        11895
        Other                    6878
        Overmedication           2187
6-11    Adverse Reaction         1752
        Accidental In